# Statistical Testing Templates for CPTAC Data

<b>Standard imports for playing with and plotting data frames.</b>

In [1]:
import pandas as pd
import numpy as np
import scipy.stats
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
import re
import seaborn as sns
import statsmodels.stats.multitest
#import altair as alt
#alt.renderers.enable('notebook') #Necessary for Altair to work

In [2]:
import CPTAC

Loading Dictionary...
Loading Clinical Data...
Loading Proteomics Data...
Loading Transcriptomics Data...
Loading CNA Data...
Loading Phosphoproteomics Data...
Loading Somatic Mutation Data...

 ******PLEASE READ******
CPTAC is a community resource project and data are made available
rapidly after generation for community research use. The embargo
allows exploring and utilizing the data, but the data may not be in a
publication until July 1, 2019. Please see
https://proteomics.cancer.gov/data-portal/about/data-use-agreement or
enter CPTAC.embargo() to open the webpage for more details.


In [15]:
somatic_mutations = CPTAC.get_somatic()
proteomics = CPTAC.get_proteomics()
phos = CPTAC.get_phosphoproteomics()

## t-test for effects of all mutations combined
<b>Note:</b> You can put any number of proteins in the list and it will compare that gene with the protein levels of all of them

### List of interacting proteins (according to STRING and Uniprot)

In [16]:
#Build the protein list; this may have only a single protein if desired
protList = ['IRS1', 'IRS2', 'RRAS', 'AKT2', 'NRAS', 'PTEN', 'AKT1', 'MRAS', 'HRAS', 'RPS6KB1', 'PIK3R1', 'PKC', 'MTOR', 'S6K', 'MAPK', 'ERBB3', 'P85A', 'P55G', 'CDK5']

In [17]:
gene = 'PIK3CA'
tested = [];
p_vals = [];
for protein in protList: 
    if protein in proteomics.columns:
        tested.append(protein)
        cross = CPTAC.compare_mutations(proteomics, protein, gene)
        cross = cross[:100]
        cross = cross[["Mutation", protein]].dropna(axis=0)
        mutated = cross.loc[cross["Mutation"] != "Wildtype"]
        wt = cross.loc[cross["Mutation"] == "Wildtype"]
        ttest = scipy.stats.ttest_ind(mutated[protein], wt[protein])
        p_vals.append(ttest[1])

In [18]:
areSignificant = statsmodels.stats.multitest.fdrcorrection(p_vals)[0]
pvals = statsmodels.stats.multitest.fdrcorrection(p_vals)[1]
significant_proteins = np.array(tested)[np.array(areSignificant)]
significant_pvals = np.array(pvals)[np.array(areSignificant)]

In [19]:
print(significant_proteins)
print(significant_pvals)

[]
[]


### Test for phosphorylation levels (mutation in any gene with phosphorylation of any protein)

In [20]:
#List of proteins (will test all phosphorylation sites on these proteins)
phosProtList = protList

In [23]:
sites = phos.columns
p_values = []
site_names = []

for protein in phosProtList:
    pattern = re.compile(protein)
    isInList = filter(pattern.search, sites)
    if next(isInList, None) is not None:
        phosphositesdf = CPTAC.compare_mutations(phos, protein, gene)
        phosphositesdf = phosphositesdf.loc[phosphositesdf['Patient_Type'] == 'Tumor'].drop('Patient_Type', axis = 1)
        for site in phosphositesdf.columns:
            if (site is not 'Mutation'):
                sitedf = CPTAC.compare_mutations(phos, site, gene)
                mutateddf = sitedf.loc[sitedf['Mutation'] != 'Wildtype'].dropna(axis=0)
                wtdf = sitedf.loc[sitedf['Mutation'] == 'Wildtype'].dropna(axis=0)
                ttest = scipy.stats.ttest_ind(mutateddf[site], wtdf[site])
                p_values.append(ttest[1])
                site_names.append(site)     
                
areSignificant = statsmodels.stats.multitest.fdrcorrection(p_values)[0]
pvalues = statsmodels.stats.multitest.fdrcorrection(p_values)[1]
significant_sites = np.array(site_names)[np.array(areSignificant)]
significant_pvalues = np.array(pvalues)[np.array(areSignificant)]

significant_vals = dict(zip(significant_sites, significant_pvalues))

print(significant_sites)
print(significant_pvalues)
print(significant_vals)

['IRS1-S1005' 'IRS1-S1100' 'IRS1-S1101' 'IRS1-S1134' 'IRS1-S270'
 'IRS1-S323' 'IRS1-S330' 'IRS1-S374' 'IRS1-S413' 'IRS1-S415' 'IRS1-S419'
 'IRS1-S421' 'IRS1-S527' 'IRS1-S531' 'IRS1-S629' 'IRS1-S672' 'IRS2-S1100'
 'IRS2-S391' 'IRS2-S560' 'IRS2-S577' 'IRS2-S915' 'IRS2-S973' 'AKT1-S126'
 'LAMTOR1-S27' 'RPS6KA3-S375' 'RPS6KA4-S682' 'RPS6KA4-S737' 'RPS6KA5-S448'
 'RPS6KC1-S449' 'RPS6KC1-S872' 'MAPK1-T181' 'MAPK1-T185' 'MAPK1-Y187'
 'MAPK14-Y182' 'MAPK3-Y204' 'MAPKAPK2-T222' 'MAPKAPK5-S212'
 'CDK5RAP2-S1061']
[0.03631657 0.00081908 0.00081908 0.0253361  0.00774579 0.01174608
 0.00976855 0.0253361  0.03486571 0.00156808 0.04694581 0.04538642
 0.02026713 0.00446233 0.00867113 0.02509466 0.00774579 0.00931712
 0.00143854 0.04165474 0.03671707 0.0253361  0.03777013 0.00551934
 0.02527061 0.00143854 0.00205041 0.03671707 0.04908903 0.01344663
 0.02527061 0.03342095 0.00823201 0.02026713 0.00446233 0.02509466
 0.01444741 0.04526765]
{'IRS1-S1005': 0.03631656835504609, 'IRS1-S1100': 0.0008190843734

### Plot phosphorylation levels and gene mutation
<b>Note:</b> There may be fewer data points due to NA values

In [ ]:
#Specify the gene and the site; you may use a string to specify the site or reference the significant results above
gene = 'PIK3CA'
site = sigPhosResults[0]

#Build the dataframe for plotting
genedf = somatic_mutations[gene].to_frame()
sites = phos.filter(regex=site)
genedf = genedf.add(sites, fill_value=0)

phos_boxplot = sns.boxplot(data=genedf, x=gene ,y=site)
phos_boxplot.set_title(gene + " gene mutation and " + site + " phosphorylation levels")
phos_boxplot = sns.stripplot(data=genedf, x=gene, y=site,jitter=True, color=".3")
phos_boxplot.set(xlabel="Somatic Gene Mutation",ylabel="Phosphoproteomics")

### Test for phosphorylation levels and protein abundance

In [ ]:
#Specify the pattern for all phosphorylation sites as well as the protein of interest
phosProt = phos.filter(regex='PIK3CA') 
protein = 'PIK3CA'
sigPhosProtResults = []

#Build dataframe with protein levels and phosphorylation levels
proteindf = proteomics[protein].to_frame()
phosProtCross = proteindf.add(phosProt, fill_value=0)

#Bonferroni correction
pcutoff = 0.05/len(cross.columns)

#Test each site for significance and print the results
for loc in phosProtCross.columns:
    if not loc == protein:
        oneSitedf = phosProtCross[[loc, protein]].dropna(axis=0)
        pearsonresult = pearsonr(oneSitedf[loc], oneSitedf[protein])
        if(pearsonresult[1] < pcutoff):
            sigPhosProtResults.append(loc)
            print("Results for " + loc)
            print(pearsonresult)

### Scatterplot of phosphorylation levels vs. protein abundance (mutation included)

In [ ]:
#Specify a protein, a gene and a site; you may use a string to specify the site or reference sigPhosProtResults above
protein = 'CTNNB1'
gene = 'CTNNB1'
site = sigPhosProtResults[1]

#Build the dataframe for plotting
protdf = proteomics[protein].to_frame()
protdfName = protein + " protein levels" #Technically only necessary when the gene and the protein have the same name
protdf.columns = [protdfName]
sites = phos.filter(regex=site)
protdf = protdf.add(sites, fill_value=0)

genedf = somatic_mutations[gene].to_frame()
genedfName = gene + " mutation"
genedf.columns = [genedfName]
finaldf = protdf.add(genedf, fill_value=0)

alt.Chart(finaldf).mark_point().encode(
    alt.X(protdfName),
    alt.Y(site),
    alt.Color(genedfName, type='nominal')
).interactive()